# Handling Missing Values

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/bushra/updated_merged_df.csv")

# Check missing values per column
missing_values = df.isnull().sum()

# Filter only those with missing data
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

# Display result
print(missing_values)


Income share held by lowest 10%                                                                          132
Income share held by highest 10%                                                                         132
Income share held by second 20%                                                                          132
Prevalence of undernourishment (% of population)                                                          84
Government expenditure on education, total (% of GDP)                                                     30
Unemployment, total (% of total labor force) (modeled ILO estimate)                                       24
Cause of death, by injury (% of total)                                                                     6
Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)      6
Cause of death, by non-communicable diseases (% of total)                                                  6
Current health expe

In [ ]:
df.drop(columns=['Income share held by lowest 10%', 'Income share held by second 20%','Income share held by highest 10%'], inplace=True)
df.head()

,Year,Country Name,Country Code,"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)","Cause of death, by injury (% of total)","Cause of death, by non-communicable diseases (% of total)",Current health expenditure (% of GDP),Diabetes prevalence (% of population ages 20 to 79),"Government expenditure on education, total (% of GDP)","Life expectancy at birth, total (years)",...,alzheimer_and_other_dementia (in thousand),%death by alzheimer_and_other_dementia,cardiovascular_diseases (in thousand),%death by cardiovascular_diseases,asthma (in thousand),%death by asthma,kidney_diseases (in thousand),%death by kidney_diseases,Income_share_bottom 40%,Income_share_top 10%
0,2000,Afghanistan,AFG,59.814650,8.175588,32.009763,9.443391,7.6,4.684761,55.005,...,0.925783,0.362498,43.481776,17.025641,2.712045,1.061923,3.038959,1.189929,14.731,34.338001
1,2010,Afghanistan,AFG,48.149914,11.030924,40.819164,8.569672,7.6,3.479450,60.702,...,1.391584,0.555581,50.096831,20.000824,2.760920,1.102279,4.287169,1.711623,15.205,33.824001
2,2015,Afghanistan,AFG,39.390981,16.389965,44.219053,10.105348,7.6,3.255800,62.270,...,1.727924,0.629098,55.987202,20.383674,2.753179,1.002370,5.218948,1.900101,15.131,34.265999
3,2019,Afghanistan,AFG,32.930451,17.288464,49.781085,14.831320,7.6,4.343190,62.941,...,2.041364,0.700211,63.310264,21.716139,2.940604,1.008661,6.418009,2.201450,14.766,34.903999
4,2020,Afghanistan,AFG,32.930451,17.288464,49.781085,15.533614,7.6,4.343190,61.454,...,2.158192,0.696604,67.264510,21.711096,3.042989,0.982191,6.869168,2.217175,14.766,34.903999


In [ ]:
import numpy as np
def impute_by_inequality(df, target_cols, top10_col, bottom40_col):
    """
    Imputes missing values in specified target columns using inequality-based group means.

    Parameters:
    - df: The DataFrame
    - target_cols: list of column names to impute
    - top10_col: name of column for income share of top 10%
    - bottom40_col: name of column for income share of bottom 40%

    Returns:
    - DataFrame with imputed values and flags
    """

    # Step 1: Create Palma Ratio and Inequality Groups
    df = df.copy()
    df['Palma Ratio'] = df[top10_col] / df[bottom40_col]

    # Remove inf or nan in Palma Ratio for binning
    df['Palma Ratio'] = df['Palma Ratio'].replace([np.inf, -np.inf], np.nan)
    df['Inequality Group'] = pd.qcut(df['Palma Ratio'], q=3, labels=['Low', 'Medium', 'High'])

    for col in target_cols:
        impute_flag = f'{col}_imputed'
        df[impute_flag] = df[col].isna()

        # Group-wise imputation
        df[col] = df.groupby('Inequality Group')[col].transform(
            lambda x: x.fillna(x.mean())
        )

    return df
target_columns = [
    'Prevalence of undernourishment (% of population)',
    'Government expenditure on education, total (% of GDP)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)',
    'Current health expenditure (% of GDP)','Cause of death, by non-communicable diseases (% of total)',
    'Cause of death, by injury (% of total)',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)'
]

df = impute_by_inequality(
    df=df,
    target_cols=target_columns,
    top10_col='Income_share_top 10%',
    bottom40_col='Income_share_bottom 40%'
)

/tmp/ipython-input-4-2358807182.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[col] = df.groupby('Inequality Group')[col].transform(
/tmp/ipython-input-4-2358807182.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[col] = df.groupby('Inequality Group')[col].transform(
/tmp/ipython-input-4-2358807182.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[col] = df.groupby('Inequality Group')[col].transform

In [ ]:
# Check missing values per column
missing_values = df.isnull().sum()

# Filter only those with missing data
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

# Display result
print(missing_values)

Series([], dtype: int64)


In [ ]:
df.to_csv("/content/drive/MyDrive/bushra/Final_clean_Datafile.csv", index=False)


In [ ]:
df.head()

,Year,Country Name,Country Code,"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)","Cause of death, by injury (% of total)","Cause of death, by non-communicable diseases (% of total)",Current health expenditure (% of GDP),Diabetes prevalence (% of population ages 20 to 79),"Government expenditure on education, total (% of GDP)","Life expectancy at birth, total (years)",...,Income_share_top 10%,Palma Ratio,Inequality Group,Prevalence of undernourishment (% of population)_imputed,"Government expenditure on education, total (% of GDP)_imputed","Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed",Current health expenditure (% of GDP)_imputed,"Cause of death, by non-communicable diseases (% of total)_imputed","Cause of death, by injury (% of total)_imputed","Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed"
0,2000,Afghanistan,AFG,59.814650,8.175588,32.009763,9.443391,7.6,4.684761,55.005,...,34.338001,2.331003,Medium,False,False,False,False,False,False,False
1,2010,Afghanistan,AFG,48.149914,11.030924,40.819164,8.569672,7.6,3.479450,60.702,...,33.824001,2.224532,Medium,False,False,False,False,False,False,False
2,2015,Afghanistan,AFG,39.390981,16.389965,44.219053,10.105348,7.6,3.255800,62.270,...,34.265999,2.264622,Medium,False,False,False,False,False,False,False
3,2019,Afghanistan,AFG,32.930451,17.288464,49.781085,14.831320,7.6,4.343190,62.941,...,34.903999,2.363809,Medium,False,False,False,False,False,False,False
4,2020,Afghanistan,AFG,32.930451,17.288464,49.781085,15.533614,7.6,4.343190,61.454,...,34.903999,2.363809,Medium,False,False,False,False,False,False,False


# Regression for dieases

# Diabetes

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/bushra/Final_clean_Datafile.csv")

In [ ]:

df.head()


,Year,Country Name,Country Code,"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)","Cause of death, by injury (% of total)","Cause of death, by non-communicable diseases (% of total)",Current health expenditure (% of GDP),Diabetes prevalence (% of population ages 20 to 79),"Government expenditure on education, total (% of GDP)","Life expectancy at birth, total (years)",...,Income_share_top 10%,Palma Ratio,Inequality Group,Prevalence of undernourishment (% of population)_imputed,"Government expenditure on education, total (% of GDP)_imputed","Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed",Current health expenditure (% of GDP)_imputed,"Cause of death, by non-communicable diseases (% of total)_imputed","Cause of death, by injury (% of total)_imputed","Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed"
0,2000,Afghanistan,AFG,59.814650,8.175588,32.009763,9.443391,7.6,4.684761,55.005,...,34.338001,2.331003,Medium,False,False,False,False,False,False,False
1,2010,Afghanistan,AFG,48.149914,11.030924,40.819164,8.569672,7.6,3.479450,60.702,...,33.824001,2.224532,Medium,False,False,False,False,False,False,False
2,2015,Afghanistan,AFG,39.390981,16.389965,44.219053,10.105348,7.6,3.255800,62.270,...,34.265999,2.264622,Medium,False,False,False,False,False,False,False
3,2019,Afghanistan,AFG,32.930451,17.288464,49.781085,14.831320,7.6,4.343190,62.941,...,34.903999,2.363809,Medium,False,False,False,False,False,False,False
4,2020,Afghanistan,AFG,32.930451,17.288464,49.781085,15.533614,7.6,4.343190,61.454,...,34.903999,2.363809,Medium,False,False,False,False,False,False,False


In [ ]:
# Step 2: Drop specified unnecessary/imputed columns
columns_to_drop = [
    'Palma Ratio',
    'Inequality Group',
    'Prevalence of undernourishment (% of population)_imputed',
    'Government expenditure on education, total (% of GDP)_imputed',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed',
    'Current health expenditure (% of GDP)_imputed',
    'Cause of death, by non-communicable diseases (% of total)_imputed',
    'Cause of death, by injury (% of total)_imputed',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed'
]
df = df.drop(columns=columns_to_drop, errors='ignore')
df.head()

,Year,Country Name,Country Code,"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)","Cause of death, by injury (% of total)","Cause of death, by non-communicable diseases (% of total)",Current health expenditure (% of GDP),Diabetes prevalence (% of population ages 20 to 79),"Government expenditure on education, total (% of GDP)","Life expectancy at birth, total (years)",...,alzheimer_and_other_dementia (in thousand),%death by alzheimer_and_other_dementia,cardiovascular_diseases (in thousand),%death by cardiovascular_diseases,asthma (in thousand),%death by asthma,kidney_diseases (in thousand),%death by kidney_diseases,Income_share_bottom 40%,Income_share_top 10%
0,2000,Afghanistan,AFG,59.814650,8.175588,32.009763,9.443391,7.6,4.684761,55.005,...,0.925783,0.362498,43.481776,17.025641,2.712045,1.061923,3.038959,1.189929,14.731,34.338001
1,2010,Afghanistan,AFG,48.149914,11.030924,40.819164,8.569672,7.6,3.479450,60.702,...,1.391584,0.555581,50.096831,20.000824,2.760920,1.102279,4.287169,1.711623,15.205,33.824001
2,2015,Afghanistan,AFG,39.390981,16.389965,44.219053,10.105348,7.6,3.255800,62.270,...,1.727924,0.629098,55.987202,20.383674,2.753179,1.002370,5.218948,1.900101,15.131,34.265999
3,2019,Afghanistan,AFG,32.930451,17.288464,49.781085,14.831320,7.6,4.343190,62.941,...,2.041364,0.700211,63.310264,21.716139,2.940604,1.008661,6.418009,2.201450,14.766,34.903999
4,2020,Afghanistan,AFG,32.930451,17.288464,49.781085,15.533614,7.6,4.343190,61.454,...,2.158192,0.696604,67.264510,21.711096,3.042989,0.982191,6.869168,2.217175,14.766,34.903999


In [ ]:
target = '%death by diabetes'
drop_cols = ['Year', 'Country Name', 'Country Code', target]
features = df.drop(columns=drop_cols, errors='ignore').columns

X = df[features]
y = df[target]

# Step 4: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Create and train a regression pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])
pipeline.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n Diabetes Regression Results")
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)



 Diabetes Regression Results
Mean Squared Error (MSE): 4.34676098901891
R² Score: 0.4082535240576117


# Alzheimer

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/bushra/Final_clean_Datafile.csv")

# Step 2: Drop specified unnecessary/imputed columns
columns_to_drop = [
    'Palma Ratio',
    'Inequality Group',
    'Prevalence of undernourishment (% of population)_imputed',
    'Government expenditure on education, total (% of GDP)_imputed',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed',
    'Current health expenditure (% of GDP)_imputed',
    'Cause of death, by non-communicable diseases (% of total)_imputed',
    'Cause of death, by injury (% of total)_imputed',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed'
]
df = df.drop(columns=columns_to_drop, errors='ignore')
df.head()
# Step 4: Define target and features
target = '%death by alzheimer_and_other_dementia'
drop_cols = ['Year', 'Country Name', 'Country Code', target]
features = df.drop(columns=drop_cols, errors='ignore').columns

X = df[features]
y = df[target]

# Step 4: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Create and train a regression pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])
pipeline.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print("\n Alzheimer Regression Results")
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)



 Alzheimer Regression Results
Mean Squared Error (MSE): 2.603699178884833
R² Score: 0.593491285760511


# Asthma

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/bushra/Final_clean_Datafile.csv")

# Step 2: Drop specified unnecessary/imputed columns
columns_to_drop = [
    'Palma Ratio',
    'Inequality Group',
    'Prevalence of undernourishment (% of population)_imputed',
    'Government expenditure on education, total (% of GDP)_imputed',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed',
    'Current health expenditure (% of GDP)_imputed',
    'Cause of death, by non-communicable diseases (% of total)_imputed',
    'Cause of death, by injury (% of total)_imputed',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed'
]
df = df.drop(columns=columns_to_drop, errors='ignore')
df.head()
# Step 4: Define target and features
target = '%death by asthma'
drop_cols = ['Year', 'Country Name', 'Country Code', target]
features = df.drop(columns=drop_cols, errors='ignore').columns

X = df[features]
y = df[target]

# Step 4: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Create and train a regression pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])
pipeline.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print("\n Asthma Regression Results")
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)



 Asthma Regression Results
Mean Squared Error (MSE): 0.1842736053438748
R² Score: 0.32373813295835807


# Kidney

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/bushra/Final_clean_Datafile.csv")

# Step 2: Drop specified unnecessary/imputed columns
columns_to_drop = [
    'Palma Ratio',
    'Inequality Group',
    'Prevalence of undernourishment (% of population)_imputed',
    'Government expenditure on education, total (% of GDP)_imputed',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed',
    'Current health expenditure (% of GDP)_imputed',
    'Cause of death, by non-communicable diseases (% of total)_imputed',
    'Cause of death, by injury (% of total)_imputed',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed'
]
df = df.drop(columns=columns_to_drop, errors='ignore')
df.head()
# Step 4: Define target and features
target = '%death by kidney_diseases'
drop_cols = ['Year', 'Country Name', 'Country Code', target]
features = df.drop(columns=drop_cols, errors='ignore').columns

X = df[features]
y = df[target]

# Step 4: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Create and train a regression pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])
pipeline.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print("\n Kidney Regression Results")
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)



 Kidney Regression Results
Mean Squared Error (MSE): 1.6912443805039181
R² Score: 0.4639059208240617


# Cardiovascular

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/bushra/Final_clean_Datafile.csv")

# Step 2: Drop specified unnecessary/imputed columns
columns_to_drop = [
    'Palma Ratio',
    'Inequality Group',
    'Prevalence of undernourishment (% of population)_imputed',
    'Government expenditure on education, total (% of GDP)_imputed',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed',
    'Current health expenditure (% of GDP)_imputed',
    'Cause of death, by non-communicable diseases (% of total)_imputed',
    'Cause of death, by injury (% of total)_imputed',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed'
]
df = df.drop(columns=columns_to_drop, errors='ignore')
df.head()
# Step 4: Define target and features
target = '%death by cardiovascular_diseases'
drop_cols = ['Year', 'Country Name', 'Country Code', target]
features = df.drop(columns=drop_cols, errors='ignore').columns

X = df[features]
y = df[target]

# Step 4: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Create and train a regression pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])
pipeline.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)



print("\n Cardiovascular Diseases Regression Results")
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)



 Cardiovascular Diseases Regression Results
Mean Squared Error (MSE): 40.36585108496807
R² Score: 0.7735666046759501


# Bottleneck for all diseases

# DIABETES

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the cleaned dataset
df = pd.read_csv("/content/drive/MyDrive/STEMCODE/clean_Datafile.csv")

# Step 1: Drop unnecessary or imputed columns
columns_to_drop = [
    'Palma Ratio',
    'Inequality Group',
    'Prevalence of undernourishment (% of population)_imputed',
    'Government expenditure on education, total (% of GDP)_imputed',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed',
    'Current health expenditure (% of GDP)_imputed',
    'Cause of death, by non-communicable diseases (% of total)_imputed',
    'Cause of death, by injury (% of total)_imputed',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed'
]
df = df.drop(columns=columns_to_drop, errors='ignore')

# Step 2: Define bottleneck concepts
bottleneck_concepts = [
    'Current health expenditure (% of GDP)',
    'Diabetes prevalence (% of population ages 20 to 79)',
    'Government expenditure on education, total (% of GDP)',
    'Life expectancy at birth, total (years)',
    'Prevalence of undernourishment (% of population)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)'
]

# Step 3: Define target and features
target = '%death by diabetes'
drop_cols = ['Year', 'Country Name', 'Country Code', target] + bottleneck_concepts
X_raw = df.drop(columns=drop_cols, errors='ignore')
C = df[bottleneck_concepts]
Y = df[target]

# Step 4: Train-test split
X_train_raw, X_test_raw, C_train, C_test, Y_train, Y_test = train_test_split(
    X_raw, C, Y, test_size=0.2, random_state=42
)

# Step 5: Normalize inputs
scaler_X = StandardScaler().fit(X_train_raw)
scaler_C = StandardScaler().fit(C_train)

X_train_scaled = scaler_X.transform(X_train_raw)
X_test_scaled = scaler_X.transform(X_test_raw)
C_train_scaled = scaler_C.transform(C_train)
C_test_scaled = scaler_C.transform(C_test)

# Step 6: Stage 1 - Predict bottleneck concepts
input_dim = X_train_scaled.shape[1]
concept_dim = len(bottleneck_concepts)

input_stage1 = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(input_stage1)
x = layers.Dense(32, activation='relu')(x)
concept_output = layers.Dense(concept_dim, name='concepts')(x)

model_stage1 = models.Model(inputs=input_stage1, outputs=concept_output)
model_stage1.compile(optimizer='adam', loss='mse')
model_stage1.fit(X_train_scaled, C_train_scaled, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# Step 7: Predict bottleneck concepts
C_train_hat = model_stage1.predict(X_train_scaled)
C_test_hat = model_stage1.predict(X_test_scaled)

# Step 8: Stage 2 - Predict final target from concepts
input_stage2 = layers.Input(shape=(concept_dim,))
x = layers.Dense(32, activation='relu')(input_stage2)
x = layers.Dense(16, activation='relu')(x)
output = layers.Dense(1, name='death_rate')(x)

model_stage2 = models.Model(inputs=input_stage2, outputs=output)
model_stage2.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_stage2.fit(C_train_hat, Y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# Step 9: Evaluate the model
loss, mae = model_stage2.evaluate(C_test_hat, Y_test)
print("CBM Test MAE for Diabetes:", round(mae, 4))


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.5906 - mae: 1.3385 
CBM Test MAE for Diabetes: 1.3607


# Alzheimer

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the cleaned dataset
df = pd.read_csv("/content/drive/MyDrive/STEMCODE/clean_Datafile.csv")

# Step 1: Drop unnecessary or imputed columns
columns_to_drop = [
    'Palma Ratio',
    'Inequality Group',
    'Prevalence of undernourishment (% of population)_imputed',
    'Government expenditure on education, total (% of GDP)_imputed',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed',
    'Current health expenditure (% of GDP)_imputed',
    'Cause of death, by non-communicable diseases (% of total)_imputed',
    'Cause of death, by injury (% of total)_imputed',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed'
]
df = df.drop(columns=columns_to_drop, errors='ignore')

# Step 2: Define bottleneck concepts
bottleneck_concepts = [
    'Current health expenditure (% of GDP)',
    'Diabetes prevalence (% of population ages 20 to 79)',
    'Government expenditure on education, total (% of GDP)',
    'Life expectancy at birth, total (years)',
    'Prevalence of undernourishment (% of population)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)'
]

# Step 3: Define target and features
target = '%death by alzheimer_and_other_dementia'
drop_cols = ['Year', 'Country Name', 'Country Code', target] + bottleneck_concepts
X_raw = df.drop(columns=drop_cols, errors='ignore')
C = df[bottleneck_concepts]
Y = df[target]

# Step 4: Train-test split
X_train_raw, X_test_raw, C_train, C_test, Y_train, Y_test = train_test_split(
    X_raw, C, Y, test_size=0.2, random_state=42
)

# Step 5: Normalize inputs
scaler_X = StandardScaler().fit(X_train_raw)
scaler_C = StandardScaler().fit(C_train)

X_train_scaled = scaler_X.transform(X_train_raw)
X_test_scaled = scaler_X.transform(X_test_raw)
C_train_scaled = scaler_C.transform(C_train)
C_test_scaled = scaler_C.transform(C_test)

# Step 6: Stage 1 - Predict bottleneck concepts
input_dim = X_train_scaled.shape[1]
concept_dim = len(bottleneck_concepts)

input_stage1 = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(input_stage1)
x = layers.Dense(32, activation='relu')(x)
concept_output = layers.Dense(concept_dim, name='concepts')(x)

model_stage1 = models.Model(inputs=input_stage1, outputs=concept_output)
model_stage1.compile(optimizer='adam', loss='mse')
model_stage1.fit(X_train_scaled, C_train_scaled, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# Step 7: Predict bottleneck concepts
C_train_hat = model_stage1.predict(X_train_scaled)
C_test_hat = model_stage1.predict(X_test_scaled)

# Step 8: Stage 2 - Predict final target from concepts
input_stage2 = layers.Input(shape=(concept_dim,))
x = layers.Dense(32, activation='relu')(input_stage2)
x = layers.Dense(16, activation='relu')(x)
output = layers.Dense(1, name='death_rate')(x)

model_stage2 = models.Model(inputs=input_stage2, outputs=output)
model_stage2.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_stage2.fit(C_train_hat, Y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# Step 9: Evaluate the model
loss, mae = model_stage2.evaluate(C_test_hat, Y_test)
print(f"\n CBM Test MAE for Alzheimer: {mae:.4f}")


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.9437 - mae: 1.0075 

 CBM Test MAE for Alzheimer: 0.9478


# Asthma

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the cleaned dataset
df = pd.read_csv("/content/drive/MyDrive/STEMCODE/clean_Datafile.csv")

# Step 1: Drop unnecessary or imputed columns
columns_to_drop = [
    'Palma Ratio',
    'Inequality Group',
    'Prevalence of undernourishment (% of population)_imputed',
    'Government expenditure on education, total (% of GDP)_imputed',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed',
    'Current health expenditure (% of GDP)_imputed',
    'Cause of death, by non-communicable diseases (% of total)_imputed',
    'Cause of death, by injury (% of total)_imputed',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed'
]
df = df.drop(columns=columns_to_drop, errors='ignore')

# Step 2: Define bottleneck concepts
bottleneck_concepts = [
    'Current health expenditure (% of GDP)',
    'Diabetes prevalence (% of population ages 20 to 79)',
    'Government expenditure on education, total (% of GDP)',
    'Life expectancy at birth, total (years)',
    'Prevalence of undernourishment (% of population)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)'
]

# Step 3: Define target and features
target = "%death by asthma"
drop_cols = ['Year', 'Country Name', 'Country Code', target] + bottleneck_concepts
X_raw = df.drop(columns=drop_cols, errors='ignore')
C = df[bottleneck_concepts]
Y = df[target]

# Step 4: Train-test split
X_train_raw, X_test_raw, C_train, C_test, Y_train, Y_test = train_test_split(
    X_raw, C, Y, test_size=0.2, random_state=42
)

# Step 5: Normalize inputs
scaler_X = StandardScaler().fit(X_train_raw)
scaler_C = StandardScaler().fit(C_train)

X_train_scaled = scaler_X.transform(X_train_raw)
X_test_scaled = scaler_X.transform(X_test_raw)
C_train_scaled = scaler_C.transform(C_train)
C_test_scaled = scaler_C.transform(C_test)

# Step 6: Stage 1 - Predict bottleneck concepts
input_dim = X_train_scaled.shape[1]
concept_dim = len(bottleneck_concepts)

input_stage1 = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(input_stage1)
x = layers.Dense(32, activation='relu')(x)
concept_output = layers.Dense(concept_dim, name='concepts')(x)

model_stage1 = models.Model(inputs=input_stage1, outputs=concept_output)
model_stage1.compile(optimizer='adam', loss='mse')
model_stage1.fit(X_train_scaled, C_train_scaled, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# Step 7: Predict bottleneck concepts
C_train_hat = model_stage1.predict(X_train_scaled)
C_test_hat = model_stage1.predict(X_test_scaled)

# Step 8: Stage 2 - Predict final target from concepts
input_stage2 = layers.Input(shape=(concept_dim,))
x = layers.Dense(32, activation='relu')(input_stage2)
x = layers.Dense(16, activation='relu')(x)
output = layers.Dense(1, name='death_rate')(x)

model_stage2 = models.Model(inputs=input_stage2, outputs=output)
model_stage2.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_stage2.fit(C_train_hat, Y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# Step 9: Evaluate the model
loss, mae = model_stage2.evaluate(C_test_hat, Y_test)
print(f"\n CBM Test MAE for Asthma: {mae:.4f}")


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1345 - mae: 0.2548 

 CBM Test MAE for Asthma: 0.2465


# Cardiovascular

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the cleaned dataset
df = pd.read_csv("/content/drive/MyDrive/STEMCODE/clean_Datafile.csv")

# Step 1: Drop unnecessary or imputed columns
columns_to_drop = [
    'Palma Ratio',
    'Inequality Group',
    'Prevalence of undernourishment (% of population)_imputed',
    'Government expenditure on education, total (% of GDP)_imputed',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed',
    'Current health expenditure (% of GDP)_imputed',
    'Cause of death, by non-communicable diseases (% of total)_imputed',
    'Cause of death, by injury (% of total)_imputed',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed'
]
df = df.drop(columns=columns_to_drop, errors='ignore')

# Step 2: Define bottleneck concepts
bottleneck_concepts = [
    'Current health expenditure (% of GDP)',
    'Diabetes prevalence (% of population ages 20 to 79)',
    'Government expenditure on education, total (% of GDP)',
    'Life expectancy at birth, total (years)',
    'Prevalence of undernourishment (% of population)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)'
]

# Step 3: Define target and features
target = "%death by cardiovascular_diseases"
drop_cols = ['Year', 'Country Name', 'Country Code', target] + bottleneck_concepts
X_raw = df.drop(columns=drop_cols, errors='ignore')
C = df[bottleneck_concepts]
Y = df[target]

# Step 4: Train-test split
X_train_raw, X_test_raw, C_train, C_test, Y_train, Y_test = train_test_split(
    X_raw, C, Y, test_size=0.2, random_state=42
)

# Step 5: Normalize inputs
scaler_X = StandardScaler().fit(X_train_raw)
scaler_C = StandardScaler().fit(C_train)

X_train_scaled = scaler_X.transform(X_train_raw)
X_test_scaled = scaler_X.transform(X_test_raw)
C_train_scaled = scaler_C.transform(C_train)
C_test_scaled = scaler_C.transform(C_test)

# Step 6: Stage 1 - Predict bottleneck concepts
input_dim = X_train_scaled.shape[1]
concept_dim = len(bottleneck_concepts)

input_stage1 = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(input_stage1)
x = layers.Dense(32, activation='relu')(x)
concept_output = layers.Dense(concept_dim, name='concepts')(x)

model_stage1 = models.Model(inputs=input_stage1, outputs=concept_output)
model_stage1.compile(optimizer='adam', loss='mse')
model_stage1.fit(X_train_scaled, C_train_scaled, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# Step 7: Predict bottleneck concepts
C_train_hat = model_stage1.predict(X_train_scaled)
C_test_hat = model_stage1.predict(X_test_scaled)

# Step 8: Stage 2 - Predict final target from concepts
input_stage2 = layers.Input(shape=(concept_dim,))
x = layers.Dense(32, activation='relu')(input_stage2)
x = layers.Dense(16, activation='relu')(x)
output = layers.Dense(1, name='death_rate')(x)

model_stage2 = models.Model(inputs=input_stage2, outputs=output)
model_stage2.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_stage2.fit(C_train_hat, Y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# Step 9: Evaluate the model
loss, mae = model_stage2.evaluate(C_test_hat, Y_test)
print(f"\n CBM Test MAE for CardioVascular Diseases: {mae:.4f}")


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 44.4814 - mae: 5.0505 

 CBM Test MAE for CardioVascular Diseases: 5.1809


# Kidney

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the cleaned dataset
df = pd.read_csv("/content/drive/MyDrive/STEMCODE/clean_Datafile.csv")

# Step 1: Drop unnecessary or imputed columns
columns_to_drop = [
    'Palma Ratio',
    'Inequality Group',
    'Prevalence of undernourishment (% of population)_imputed',
    'Government expenditure on education, total (% of GDP)_imputed',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)_imputed',
    'Current health expenditure (% of GDP)_imputed',
    'Cause of death, by non-communicable diseases (% of total)_imputed',
    'Cause of death, by injury (% of total)_imputed',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)_imputed'
]
df = df.drop(columns=columns_to_drop, errors='ignore')

# Step 2: Define bottleneck concepts
bottleneck_concepts = [
    'Current health expenditure (% of GDP)',
    'Diabetes prevalence (% of population ages 20 to 79)',
    'Government expenditure on education, total (% of GDP)',
    'Life expectancy at birth, total (years)',
    'Prevalence of undernourishment (% of population)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)'
]

# Step 3: Define target and features
target = '%death by kidney_diseases'
drop_cols = ['Year', 'Country Name', 'Country Code', target] + bottleneck_concepts
X_raw = df.drop(columns=drop_cols, errors='ignore')
C = df[bottleneck_concepts]
Y = df[target]

# Step 4: Train-test split
X_train_raw, X_test_raw, C_train, C_test, Y_train, Y_test = train_test_split(
    X_raw, C, Y, test_size=0.2, random_state=42
)

# Step 5: Normalize inputs
scaler_X = StandardScaler().fit(X_train_raw)
scaler_C = StandardScaler().fit(C_train)

X_train_scaled = scaler_X.transform(X_train_raw)
X_test_scaled = scaler_X.transform(X_test_raw)
C_train_scaled = scaler_C.transform(C_train)
C_test_scaled = scaler_C.transform(C_test)

# Step 6: Stage 1 - Predict bottleneck concepts
input_dim = X_train_scaled.shape[1]
concept_dim = len(bottleneck_concepts)

input_stage1 = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(input_stage1)
x = layers.Dense(32, activation='relu')(x)
concept_output = layers.Dense(concept_dim, name='concepts')(x)

model_stage1 = models.Model(inputs=input_stage1, outputs=concept_output)
model_stage1.compile(optimizer='adam', loss='mse')
model_stage1.fit(X_train_scaled, C_train_scaled, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# Step 7: Predict bottleneck concepts
C_train_hat = model_stage1.predict(X_train_scaled)
C_test_hat = model_stage1.predict(X_test_scaled)

# Step 8: Stage 2 - Predict final target from concepts
input_stage2 = layers.Input(shape=(concept_dim,))
x = layers.Dense(32, activation='relu')(input_stage2)
x = layers.Dense(16, activation='relu')(x)
output = layers.Dense(1, name='death_rate')(x)

model_stage2 = models.Model(inputs=input_stage2, outputs=output)
model_stage2.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_stage2.fit(C_train_hat, Y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# Step 9: Evaluate the model
loss, mae = model_stage2.evaluate(C_test_hat, Y_test)
print(f"\n CBM Test MAE for Kidney Diseases: {mae:.4f}")


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.1782 - mae: 0.7242 

 CBM Test MAE for Kidney Diseases: 0.7485
